# Парсинг

In [ ]:
# импорт библиотек
import requests # для http запросов
import pandas as pd # для датафреймов
from bs4 import BeautifulSoup # парсинг html
from random import choice # случайный выбор из списка
from time import sleep # парсер должен останавливаться на паузы, чтобы не нагружать систему и не забанили адрес
from tqdm import tqdm # прогрессбар

In [ ]:
home_page = 'http://citystar.ru/'
sleep_time = range(10, 30) # малая пауза для остановки
grand_sleep = range(600, 900) # большая пазуа для остановки

# формат запроса берем из браузера
params = {
    "credentials": "include",
    "headers": {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Upgrade-Insecure-Requests": "1"
    },
    "referrer": "http://citystar.ru/",
    "method": "GET",
    "mode": "cors"
}

# функция HTTP запроса
def fetch(url, params, ref):
  params['referrer'] = ref # обновим заголовок refferer
  headers = params['headers']
  body = params.get('body')
  if params["method"] == 'GET':
    r = requests.get(url, headers=headers)
  else:
    r = requests.post(url, headers=headers, data=body)
  # декодируем текст из кодировки 'cp1251' в строку
  return r.content.decode('cp1251') #.encode('utf8')

# функция подзапроса типа планировки с карточки обьекта
def get_planning_type(url, params, ref):
  r = fetch(url, params, ref) # скачаем карточку объекта
  s = BeautifulSoup(r) # создаем суп
  res = ''
  try:
    res = s.find_all('td', {'class': 'frc'})[8].text.replace(u'\xa0', u'') # убрать неразрывной пробел
  except:
    pass
  return res

# x = get_planning_type(home_page+'/scard.asp?d=43&id=14954724', params, home_page+'detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF%20-%20%CF%F0%EE%E4%E0%EC%20%EA%E2%E0%F0%F2%E8%F0%F3%20%E2%20%E3.%20%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5')
# x

# функция возвращает url на следующую страницу 
def get_next_page(text):
  try:
    pages_soup = BeautifulSoup(text) # суп для поиска линков на другие страницы
    pages = pages_soup.find_all('td', {'class': 'lstb'}) # все доступные линки на страницы
    page_count = len(pages[1].find_all('a', {'class': 'link'}))
    for i in range(len(pages[1].find_all('a', {'class': 'link'}))): # перебираем линки
      if pages[1].find_all('a', {'class': 'link'})[i].find('b')!=None: # если текущий линк помечен тэгом <b></b>, значит следущий линк будет следующей страницей 
        break
    if i == page_count - 1: # если в списке линков на другие страницы это последний
      res = '' # то вернем пустой url
    else:
      res = pages[1].find_all('a', {'class': 'link'})[i+1].get('href') # иначе вернем следущий линк из списка линков
  except:
    res = ''
  return res

# detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2
# следующая страница
# url = home_page + 'detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2'
# url = 'http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=1'
# text = fetch(url, params)
# print(get_next_page(text))

# функция для парсинга страницы
def parse_page(url, params, ref):
  new_data = [] # здесь будет таблица с данными по каждой карточке

  text = fetch(url, params, ref) # загрузим страницу

  # парсим с помощью Beatifull Soup
  soup = BeautifulSoup(text) # загрузим text в суп
  rows = soup.find_all('tr', {'class': 'tbb'}) # получим все карточки 
  for i in tqdm(range(len(rows))):
    # в строке 29 элементов и 14 колонок
    if len(rows[i])!=29: continue # строки без обьявлений пропускаем
    # print(len(rows[i])) 
    cols = rows[i].find_all('td') # берем колонки из строки
    if len(cols)!=14: continue # если колонок не 14 пропускаем
    # print(len(cols)) 
  
    # инициализируем значения перед каждой карточкой
    photo_url = date = apart_type = district = address = floor = area1 = area2 = area3 = planning = comment = price = phone = name = agency = email = ''

    # ссылка на фото
    try:
      photo_url = home_page + cols[0].find('a').find('img').get('src')
    except:
      pass

    # планировка объекта
    try:
      planning = get_planning_type(home_page + cols[0].find('a')['href'], params, url)
    except:
      pass

    # дата
    date = cols[1].text[:5] 

    # тип апартаментов
    apart_type = cols[2].text

    # район
    district = cols[3].text

    # адрес
    address = cols[4].text

    # этаж
    floor = cols[5].text

    # площадь общая
    area1 = cols[6].text

    # площадь жилая
    area2 = cols[7].text

    # площадь кухня
    area3 = cols[8].text

    # комментарий
    comment = cols[9].text.replace('\r\n', ' ')

    # цена в тыс. руб.
    price = cols[10].text

    # телефон и имя
    try:
      phone = cols[11].text.split('...')[0].strip()
      name = cols[11].text.split('...')[1].strip()
    except:
      pass

    # агентство
    agency = cols[12].text

    # email
    email = cols[13].text
  
    # добавим строку в таблицу по объекту
    new_data.append([photo_url, planning, date, apart_type, district, address, floor, area1, area2, area3, comment, price, phone, name, agency, email])

    # взять следующую страницу
    next_page = get_next_page(text)

    # малая пауза
    # sleep(choice(sleep_time))

  return new_data, next_page

def main():
  data = [] # собираем данные в эту таблицу
  next_ref = home_page # инициализируем refferal параметр
  next_page = "detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF%20-%20%CF%F0%EE%E4%E0%EC%20%EA%E2%E0%F0%F2%E8%F0%F3%20%E2%20%E3.%20%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5"
  
  while next_page != '': # парсим пока есть следующая страница 
    url = home_page + next_page # соберем url перед парсингом очередной страницей
    print(f'Парсим: {url}')
    new_data, next_page = parse_page(url, params, next_ref) # парсим страницу
    next_ref = url # обновим refferal параметр
    data.extend(new_data) # добавим новые записи в общий список
    # sleep(choice(grand_sleep)) # большая пауза
 
  return data

In [ ]:
data = main()

Парсим: http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF%20-%20%CF%F0%EE%E4%E0%EC%20%EA%E2%E0%F0%F2%E8%F0%F3%20%E2%20%E3.%20%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5


100%|██████████| 102/102 [03:15<00:00,  1.92s/it]


Парсим: http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2


100%|██████████| 102/102 [03:07<00:00,  1.84s/it]


Парсим: http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=3


...

100%|██████████| 102/102 [02:18<00:00,  1.35s/it]


Парсим: http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=8


100%|██████████| 102/102 [02:55<00:00,  1.72s/it]


Парсим: http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=9


100%|██████████| 82/82 [02:26<00:00,  1.79s/it]


In [ ]:
print(len(data[0]))
data[:1]

16


[['http://citystar.ru/img.aspx?/fs/mq/9bd3ff0093942edbcaeaddcc8b08bb2d.jpg',
  'раздельная',
  '13.05',
  'Трехкомнатная раздельная',
  'Ленинский',
  'Куйбышева 12',
  '1/3',
  '57.00',
  '35.00',
  '8.00',
  'Трехкомнатная квартира нестандартной планировки (третья комната с отдельным входом) в нормальном состоянии, санузел раздельный, трубы пластиковые, водомеры. Первый этаж, высоко. В центре Ленинского района. Есть все: садики, школы, скверы и парки, магазины, поликлиника. Очень тихое, спокойное место.',
  '2850',
  '+79615797796',
  'Дедевич Алексей Константинович',
  'Выбор',
  'vubor34@mail.ru']]

In [ ]:
 # создаем датафрейм с данными
df = pd.DataFrame(data=data, columns = ['photo_url', 'planning', 'date', 'apart_type', 'district', 'address', 'floor', 'area1', 'area2', 'area3', 'comment', 'price', 'phone', 'name', 'agency', 'email'])

In [ ]:
df.head() # первые 5 записей

,photo_url,planning,date,apart_type,district,address,floor,area1,area2,area3,comment,price,phone,name,agency,email
0,http://citystar.ru/img.aspx?/fs/mq/9bd3ff00939...,раздельная,13.05,Трехкомнатная раздельная,Ленинский,Куйбышева 12,1/3,57.00,35.00,8.00,Трехкомнатная квартира нестандартной планировк...,2850,+79615797796,Дедевич Алексей Константинович,Выбор,vubor34@mail.ru
1,http://citystar.ru/img.aspx?/fs/0c/30bfcca8a96...,нестандартная,12.05,Трехкомнатная нестандартная,Орджоникидзевский,50-летия Магнитки 31,3/9,98.50,60.00,15.00,Продам трехкомнатную квартиру 100кв.м. Простор...,6990,8 951 235 77 55,Климов Евгений,"""Шанс""",agshans@mail.ru
2,http://citystar.ru/img.aspx?/fs/0d/c4a36e3da97...,старой планировки,19.05,Трехкомнатная старой планировки,Ленинский,Менделеева 6,3/4,83.00,55.00,9.00,ПРОДАМ трехкомнатную квартиру старой планировк...,5100,89128051276,Нестерова Айгуль Зинуровна,,
3,http://citystar.ru/img.aspx?/fs/g0/097f4353fdb...,малосемейка,19.05,Однокомнатная малосемейка,Правобережный,Ленина пр-т 98/1,7/9,30.40,17.00,6.00,Квартира малосемейка в хорошем состоянии. П...,1790,89128051254,Байгузина Юлия Владимировна,Визит,
4,http://citystar.ru/img.aspx?/fs/0k/b06825c0aed...,,19.05,Трехкомнатная,Ленинский,Ленина пр-т 10,5/5,77.00,48.00,9.00,КВАРТИРА СТАРОЙ ПЛАНИРОВКИ С ВЫСОКИМИ ПОТОЛКА...,3990,89128051254,Байгузина Юлия Владимировна,Визит,


In [ ]:
df.info() # информация по датафрейму

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   photo_url   880 non-null    object
 1   planning    880 non-null    object
 2   date        880 non-null    object
 3   apart_type  880 non-null    object
 4   district    880 non-null    object
 5   address     880 non-null    object
 6   floor       880 non-null    object
 7   area1       880 non-null    object
 8   area2       880 non-null    object
 9   area3       880 non-null    object
 10  comment     880 non-null    object
 11  price       880 non-null    object
 12  phone       880 non-null    object
 13  name        880 non-null    object
 14  agency      880 non-null    object
 15  email       880 non-null    object
dtypes: object(16)
memory usage: 110.1+ KB


In [ ]:
# df.to_csv('citystar.csv', index=False)

In [ ]:
df = pd.read_csv('citystar.csv')

In [ ]:
df.head()

,photo_url,planning,date,apart_type,district,address,floor,area1,area2,area3,comment,price,phone,name,agency,email
0,http://citystar.ru/img.aspx?/fs/mq/9bd3ff00939...,раздельная,13.05,Трехкомнатная раздельная,Ленинский,Куйбышева 12,1/3,57.0,35.0,8.0,Трехкомнатная квартира нестандартной планировк...,2850,+79615797796,Дедевич Алексей Константинович,Выбор,vubor34@mail.ru
1,http://citystar.ru/img.aspx?/fs/0c/30bfcca8a96...,нестандартная,12.05,Трехкомнатная нестандартная,Орджоникидзевский,50-летия Магнитки 31,3/9,98.5,60.0,15.0,Продам трехкомнатную квартиру 100кв.м. Простор...,6990,8 951 235 77 55,Климов Евгений,"""Шанс""",agshans@mail.ru
2,http://citystar.ru/img.aspx?/fs/0d/c4a36e3da97...,старой планировки,19.05,Трехкомнатная старой планировки,Ленинский,Менделеева 6,3/4,83.0,55.0,9.0,ПРОДАМ трехкомнатную квартиру старой планировк...,5100,89128051276,Нестерова Айгуль Зинуровна,NaN,NaN
3,http://citystar.ru/img.aspx?/fs/g0/097f4353fdb...,малосемейка,19.05,Однокомнатная малосемейка,Правобережный,Ленина пр-т 98/1,7/9,30.4,17.0,6.0,Квартира малосемейка в хорошем состоянии. П...,1790,89128051254,Байгузина Юлия Владимировна,Визит,NaN
4,http://citystar.ru/img.aspx?/fs/0k/b06825c0aed...,NaN,19.05,Трехкомнатная,Ленинский,Ленина пр-т 10,5/5,77.0,48.0,9.0,КВАРТИРА СТАРОЙ ПЛАНИРОВКИ С ВЫСОКИМИ ПОТОЛКА...,3990,89128051254,Байгузина Юлия Владимировна,Визит,NaN


In [ ]:
df.iloc[0,10] # столбец комментария

'Трехкомнатная квартира нестандартной планировки (третья комната с отдельным входом) в нормальном состоянии, санузел раздельный, трубы пластиковые, водомеры. Первый этаж, высоко. В центре Ленинского района. Есть все: садики, школы, скверы и парки, магазины, поликлиника. Очень тихое, спокойное место.'

In [ ]:
df.info() # структура датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   photo_url   818 non-null    object 
 1   planning    198 non-null    object 
 2   date        880 non-null    float64
 3   apart_type  880 non-null    object 
 4   district    695 non-null    object 
 5   address     880 non-null    object 
 6   floor       880 non-null    object 
 7   area1       880 non-null    float64
 8   area2       880 non-null    float64
 9   area3       880 non-null    float64
 10  comment     872 non-null    object 
 11  price       880 non-null    int64  
 12  phone       874 non-null    object 
 13  name        880 non-null    object 
 14  agency      502 non-null    object 
 15  email       667 non-null    object 
dtypes: float64(4), int64(1), object(11)
memory usage: 110.1+ KB


In [ ]:
# после пересохранения в CSV вернем всем полям тип строки, 
# т.к. на inferenсe модели и перед обучением будем парсить заново и переводить в категориальные данные
for col in df.columns:
  df[col] = df[col].astype('str')

In [ ]:
df['comment_len'] = [len(df.iloc[i,10]) for i in range(df.shape[0])] # посмотрим на длину комментариев
df['comment_len'].max() # длина максимального комментария (это поле в базе сделаем типом varchar(3000))

2540

In [ ]:
df.drop(columns='comment_len', axis=1, inplace=True) # удалим столбец
# df.info()

In [ ]:
# посмотрим на длины столбцов чтобы сохранить в строковом формате все столбцы (нужно для дальнейшего анализа данных)
df_len = df.copy() # сделаем копию датафрейма
col_count = len(df_len.columns) # сохраним количество столбцов
for j in range(len(df_len.columns)): # проходим по столбцам
  df_len[df_len.columns[j]] = [len(df_len.iloc[i,j]) for i in range(df_len.shape[0])] # берем длину строк и заменяем данные на длины
  print(f'{df_len.columns[j]} : {df_len[df_len.columns[j]].max()}') # найдем максимальные по столбцам

photo_url : 73
planning : 21
date : 5
apart_type : 35
district : 31
address : 27
floor : 5
area1 : 5
area2 : 5
area3 : 5
comment : 2540
price : 5
phone : 64
name : 33
agency : 43
email : 35


In [ ]:
'''
Создаем таблицу в PostgreSQL со следующей СХЕМОЙ ДАННЫХ:
---------------------------------------------
CREATE TABLE citystar (
id SERIAL PRIMARY KEY,
photo_url varchar(100),
planning varchar(30),
date varchar(10),
apart_type varchar(50),
district varchar(50),
address varchar(50),
floor varchar(10),
area1 varchar(10),
area2 varchar(10),
area3 varchar(10),
comment varchar(3000),
price varchar(10),
phone varchar(100),
name varchar(50),
agency varchar(50),
email varchar(50)
);
'''

In [ ]:
df.head(1)

,photo_url,planning,date,apart_type,district,address,floor,area1,area2,area3,comment,price,phone,name,agency,email
0,http://citystar.ru/img.aspx?/fs/mq/9bd3ff00939...,раздельная,13.05,Трехкомнатная раздельная,Ленинский,Куйбышева 12,1/3,57.0,35.0,8.0,Трехкомнатная квартира нестандартной планировк...,2850,+79615797796,Дедевич Алексей Константинович,Выбор,vubor34@mail.ru


In [ ]:
# выгрузим все в базу PostgreSQL
import pandas as pd # пандас
from sqlalchemy import create_engine # движок для субд

# параметры подключения 
db_config = {
    'user': 'ufamax',
    'pwd': 'password',
    'host': 'm.ufamax.eu',
    'port': 5432,
    'db': 'ufamax',
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)
 
engine = create_engine(connection_string)
 
# query = ''' SELECT * FROM citystar '''
 
# data_raw = pd.io.sql.read_sql(query, con=engine, index_col='id')
df.to_sql(name='citystar', con=engine, if_exists='replace', index=False) # сохранить в БД

880

In [ ]:
# !pip install --upgrade 'sqlalchemy<2.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# загрузим из базы PostgreSQL датасет
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text # нужен для обхода ошибки с PostgreSQL и sqlalchemy
 
# параметры подключения 
db_config = {
    'user': 'ufamax',
    'pwd': 'password',
    'host': 'm.ufamax.eu',
    'port': 5432,
    'db': 'ufamax',
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)
 
engine = create_engine(connection_string)
 
query = ''' SELECT * FROM citystar '''
 
with engine.begin() as conn:
    query = text(query) # текстовый запрос оборачиваем методом text()
    data_raw = pd.read_sql_query(query, conn)

# data_raw = pd.io.sql.read_sql(query, con=engine) # МЕТОД НЕ РАБОТАЕТ c PostgreSQL из-за последнего обновления sqlalchemy
# df.to_sql(name='citystar', con=engine, if_exists='replace', index=False) # сохранить в БД

data_raw.head() # первые пять

,photo_url,planning,date,apart_type,district,address,floor,area1,area2,area3,comment,price,phone,name,agency,email
0,http://citystar.ru/img.aspx?/fs/mq/9bd3ff00939...,раздельная,13.05,Трехкомнатная раздельная,Ленинский,Куйбышева 12,1/3,57.0,35.0,8.0,Трехкомнатная квартира нестандартной планировк...,2850,+79615797796,Дедевич Алексей Константинович,Выбор,vubor34@mail.ru
1,http://citystar.ru/img.aspx?/fs/0c/30bfcca8a96...,нестандартная,12.05,Трехкомнатная нестандартная,Орджоникидзевский,50-летия Магнитки 31,3/9,98.5,60.0,15.0,Продам трехкомнатную квартиру 100кв.м. Простор...,6990,8 951 235 77 55,Климов Евгений,"""Шанс""",agshans@mail.ru
2,http://citystar.ru/img.aspx?/fs/0d/c4a36e3da97...,старой планировки,19.05,Трехкомнатная старой планировки,Ленинский,Менделеева 6,3/4,83.0,55.0,9.0,ПРОДАМ трехкомнатную квартиру старой планировк...,5100,89128051276,Нестерова Айгуль Зинуровна,nan,nan
3,http://citystar.ru/img.aspx?/fs/g0/097f4353fdb...,малосемейка,19.05,Однокомнатная малосемейка,Правобережный,Ленина пр-т 98/1,7/9,30.4,17.0,6.0,Квартира малосемейка в хорошем состоянии. П...,1790,89128051254,Байгузина Юлия Владимировна,Визит,nan
4,http://citystar.ru/img.aspx?/fs/0k/b06825c0aed...,nan,19.05,Трехкомнатная,Ленинский,Ленина пр-т 10,5/5,77.0,48.0,9.0,КВАРТИРА СТАРОЙ ПЛАНИРОВКИ С ВЫСОКИМИ ПОТОЛКА...,3990,89128051254,Байгузина Юлия Владимировна,Визит,nan


In [ ]:
# структура данных
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   photo_url   880 non-null    object
 1   planning    880 non-null    object
 2   date        880 non-null    object
 3   apart_type  880 non-null    object
 4   district    880 non-null    object
 5   address     880 non-null    object
 6   floor       880 non-null    object
 7   area1       880 non-null    object
 8   area2       880 non-null    object
 9   area3       880 non-null    object
 10  comment     880 non-null    object
 11  price       880 non-null    object
 12  phone       880 non-null    object
 13  name        880 non-null    object
 14  agency      880 non-null    object
 15  email       880 non-null    object
dtypes: object(16)
memory usage: 110.1+ KB


In [ ]:
data_raw.to_csv('citystar.csv', index=False) # сохраним датасет для анализа, парстинга и создания модели